# 🔴 Lesson 12: Memory Management 101

Welcome to **Module 4: Engineering & Optimization**. This is where we stop being artists and start being engineers.

The #1 crash reason in AI is **CUDA Out of Memory (OOM)**.

### Goals:
1.  Inspect VRAM usage.
2.  See the cost of 512x512 vs 1024x1024.
3.  Learn to `gc.collect()`.

In [ ]:
# 1. Setup
import notebook_utils
project_root, device, dtype = notebook_utils.setup_notebook()

import torch
import gc
from core.pipeline import pipeline_manager

## 1. Tracking VRAM

Since we are using DirectML (AMD), standard PyTorch VRAM tools might not work perfectly, but we can track RAM.

In [ ]:
import psutil
import os

def print_memory():
    process = psutil.Process(os.getpid())
    print(f"System RAM Used by Python: {process.memory_info().rss / 1024 / 1024:.2f} MB")

print_memory()

## 2. Load the Model (Heavy!)
Watch the RAM spike.

In [ ]:
pipe = pipeline_manager.get_txt2img_pipeline()
print("Model Loaded.")
print_memory()

## 3. Resolution Cost

VRAM usage scales **quadratically** with image size (Area = $W \times H$).

- 512x512 = 262,144 pixels
- 1024x1024 = 1,048,576 pixels (**4x larger!**)

Doubling the resolution quadruples the memory cost. Be careful.

## 4. The Cleanup

When you delete a variable in Python (`del pipe`), the memory isn't freed immediately. You must call the Garbage Collector.

In [ ]:
del pipe
print("Variable deleted (Memory normally stays high)...")
print_memory()

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("Garbage Collected.")
print_memory()